In [ ]:
import DataSetQuery
import AsyncDataSetQuery
import datetime
import MalardHelpers

environmentName = 'DEVv2'
#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000',environmentName)
queryAsync = AsyncDataSetQuery.AsyncDataSetQuery('ws://localhost:9000',environmentName)

print(query.getEnvironment(environmentName))

#determines the databases that store multiple datasets 
print(query.getParentDataSets())
#retrieve the data sets for a given parent
print(query.getDataSets('cryotempo'))
#The overall data cube for the two datasets

parentDs = 'cryotempo'
dataSet = 'oib'
region='greenland'

print(query.getDataSetBoundingBox( parentDs,dataSet,region ))
#print(query.getDataSetBoundingBox( 'mtngla','srtm','himalayas' ))

#Setup the bounding box
minX=-800000
maxX=-700000
minY=-900000
maxY=-800000
minT=datetime.datetime.fromtimestamp(1461068785)
maxT=datetime.datetime.fromtimestamp(1461068785)

#print( datetime.datetime.timestamp(minT))
#print( datetime.datetime.timestamp(maxT))

#Queries the catalogue and returns the gridcells grouped by x and y with the min and max times.
#gcs = query.getGridCells(  'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT )
#print("Found [%d] Grid cells" % ( len(gcs)) )
#print(gcs)
#Queries the catalogue to determine the shards that comprise the bounding box of interest.
#shards = query.getShards( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT)
#print(shards )





In [ ]:
# loads the json output of the getGridCells call and creates a flattened DataFrame.
from pandas.io.json import json_normalize
import json

data = json.loads(gcs)
df = json_normalize(data)

print(df)

#totalPoints = df['totalPoints'].sum()
#maxNumPoints = df['totalPoints'].max()

#print('TP=[%d] MaxPoints in Cell=[%d]'%(totalPoints,maxNumPoints))   



In [ ]:
#Converts the Json output of the getShards call into a DataFrame.
dataShard = json.loads(shards)
dfShards = json_normalize(dataShard)

print(dfShards['shardName'])


In [ ]:

#a list of columns to see in the output (x,y,time will be added)
projections = ['elev','power','coh'] 
#a list of dictionaries that specify the filters
#only numeric fields are supported with the following operations gt(greater than), gte ( greater than equals), lt (less than) and lte (less than equals)
#filters are treated as and conditions
filters = [{'column':'power','op':'gt','threshold':10000},{'column':'coh','op':'gt','threshold':0.8}]

#No filters
nofilters = queryAsync.executeQuery(  'cryotempo','oib','greenland',minX,maxX,minY,maxY,minT,maxT,[],[])
#Converts the NetCDF into a DataFrame.
dfNoFilter = MalardHelpers.getDataFrameFromNetCDF(nofilters.resultFileName)

print(dfNoFilter)

#print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( dfNoFilter['coh'].max(),  dfNoFilter['coh'].min(), dfNoFilter['power'].max(), dfNoFilter['power'].min(), dfNoFilter['power'].count() ) )  

#With filters
#withfilters = query.executeQuery( 'mtngla','tdx','himalayas',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
#withFilter = MalardHelpers.getDataFrameFromNetCDF(withfilters)

#print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['coh'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  

#print( "Max Coh [%f] Min Coh [%f] Max Power [%f] Min Power [%f] Count [%d]" % ( withFilter['coh'].max(),  withFilter['coh'].min(), withFilter['power'].max(), withFilter['power'].min(), withFilter['power'].count() ) )  


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
## plot with no filters
plt.figure(figsize=(12,10))
plt.scatter(x=dfNoFilter['x'], y=dfNoFilter['y'], c=dfNoFilter['WGS84_Ellipsoid_Height(m)'], marker='.', s=20, vmax=7500)
plt.colorbar()
plt.show()

In [ ]:
## plot with filters
plt.figure(figsize=(12,10))
#vmin and vmax set min and max of colour bar
plt.scatter(x=withFilter['x'], y=withFilter['y'], c=withFilter['elev'], marker='.', s=20, vmax=dfNoFilter['elev'].max(), vmin=dfNoFilter['elev'].min())
plt.colorbar()
plt.show()

In [ ]:

#swathDetails - information about the source swath data and how the DataSetLoader did the gridding.
results = query.getSwathDetails('mtngla','tdx','himalayas')

print(results)


In [ ]:
swathById = query.getSwathDetailsFromId('mtngla','tandemx','himalayas',1)

print(swathById)
 
swathByName = query.getSwathDetailsFromName('mtngla','tdx','himalayas','CS_LTA__SIR_SIN_2S_20100716T075344_20100716T075554_C001.nc')

print(swathByName)


In [ ]:
#swathDetails - get all the swaths for a particular data set.
import pandas as pd
import MalardHelpers as m

df = m.getSwathDetailsAsDataFrame('mtngla','tdx','himalayas',query)

print(df[0:1])


In [ ]:
import json
swathDetails = json_normalize(json.loads(query.getSwathDetails('mtngla','tdx','himalayas')))

print(len(swathDetails['swathName']))


In [ ]:
sourcePath = '/data/snail1/scratch/test/'#put your input path in here
fileName = 'testmask.shp'#put your test file in here
parentDsName = 'test'
dataSet = 'RS6'
maskType = 'Glacier' 
region = 'HMA' 
minX = 500000 
minY = 0 
size = 100000 

query.publishMask(sourcePath, fileName, parentDsName, dataSet, maskType, region, minX, minY, size )

In [ ]:
print(query.getMasks('mtngla'))

In [ ]:
parentdataset = 'mtngla'
dataSet = 'RGIv60'
maskType = 'Glacier'
region = 'HMA'

print(query.getGridCellMasks(parentdataset, dataSet, maskType, region))

In [ ]:
#gets the mask for a gridcell.

print(query.getGridCellMask(parentdataset, dataSet, maskType, region,minX,minY,size))

In [ ]:
parentDsName = 'test'
runName = 'gctest_2'
minX = 0.0
minY = 200000.0
size = 100000.0

#expects a dictionary of string to double.
statistics = {'NumberOfPoints':1000,'MaskedPoints':500}

query.publishGridCellStats( parentDsName, runName, minX, minY, size, statistics )

In [ ]:
#get the available run statistics
parentDsName = 'mtngla'
query.getAvailableRunStatistics( parentDsName )

In [ ]:
#get the statistics for a gridcell.
query.getGridCellStatistics(parentDsName, runName, minX, minY, size)

In [ ]:
query.getRunStatistics(parentDsName,"TestRun")

In [ ]:
#Get the proj4 for code for the given shortCode.
print( query.getProjection('HMA'))

In [ ]:
#Publish points for a grid cell.
parentDataSet = 'mtngla' #intended usage mtngla
dataSet = 'publishtest'
region = 'himalayas'
projection = 'HMA'
sourceFileName = '/data/snail1/scratch/tandemx/swath/y2017/m6/cell_HMA_0_600000/GridCell_0.nc'
minX = 0
minY = 600000
size = 100000

query.publishGridCellPoints(parentDataSet, dataSet,region, minX, minY, size, sourceFileName, projection)


In [ ]:
print(datetime.datetime.fromtimestamp(1337598197))
print(datetime.datetime.fromtimestamp(1279356087))